In [4]:
import os
from matplotlib import pyplot as plt
import matplotlib.path as mpath
import numpy as np
import pandas as pd
import xarray as xr
import cartopy
%matplotlib inline
plt.rcParams['figure.figsize'] = 12, 6
%config InlineBackend.figure_format = 'retina' 

In [7]:
! ls ../data

MARcst-AN35km-176x148.cdf            year-MAR_NorESM-1980-2100_zen.nc2
year-MAR_ACCESS1.3-1980-2100_zen.nc2 year_ACCESS1-3.nc2
year-MAR_CESM2-1980-2100_zen.nc2     year_CESM2.nc2
year-MAR_CNRM-CM6-1980-2100_zen.nc2  year_CNRM-CM6-1.nc2
year-MAR_ERA5-1979-2019_zen.nc2      year_NorESM1-M.nc2


In [8]:
dsr = xr.open_dataset('../data/year-MAR_ACCESS1.3-1980-2100_zen.nc2')
dl = dsr.load()
dl

<xarray.Dataset>
Dimensions:    (X: 176, Y: 148, SECTOR1_1: 1, TIME: 121, bnds: 2)
Coordinates:
  * X          (X) float32 -3.01e+03 -2.975e+03 -2.94e+03 ... 3.08e+03 3.115e+03
  * Y          (Y) float32 -2.555e+03 -2.52e+03 ... 2.555e+03 2.59e+03
  * SECTOR1_1  (SECTOR1_1) float32 1.0
  * TIME       (TIME) datetime64[ns] 1980-07-02 ... 2100-07-02T12:00:00
Dimensions without coordinates: bnds
Data variables:
    TIME_bnds  (TIME, bnds) datetime64[ns] 1980-01-01 1981-01-01 ... 2101-01-01
    SMB        (TIME, SECTOR1_1, Y, X) float64 nan nan nan nan ... nan nan nan
    SF         (TIME, Y, X) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    RF         (TIME, Y, X) float32 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    SU         (TIME, SECTOR1_1, Y, X) float32 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RU         (TIME, SECTOR1_1, Y, X) float32 inf inf inf inf ... inf inf inf
Attributes:
    Conventions:       CF-1.6
    NCO:               netCDF Operators version 4.8.1 (Homepage = http://nco....
    contact:           ckittel@uliege.be
    creation_date:     2021-01-23-T13:24:17Z
    driving_model_id:  ACCESS1.3
    institute_id:      ULg
    institution:       University of Liege - Belgium
    reference:         http://www.climato.be
    tracking_id:       ddbf4239bdd46d1127a1bf5625cf2bc0

In [9]:
mask2d= dl.masker.values
lon2d = dl.masker.lon2d.reshape(dl.masker.shape)
lat2d = dl.masker.lat2d.reshape(dl.masker.shape)

AttributeError: 'Dataset' object has no attribute 'masker'